In [1]:
# Install the packages
!pip install tabpfn
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/four_classification_smote


classification_input.csv


In [5]:
import torch
import pandas as pd
from tabpfn import TabPFNClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from tabpfn import TabPFNClassifier
import pickle
from imblearn.over_sampling import SMOTE

DATA_PATH = "/content/drive/MyDrive/four_classification_smote/classification_input.csv"
MODEL_PATH = "/content/drive/MyDrive/four_classification_smote/tabpfn_classifier_4class.pkl"
FIANL_RESULTS_PATH = "/content/drive/MyDrive/four_classification_smote/test_result_TabPFN_four_classification.csv"

# Load data
data = pd.read_csv(DATA_PATH)
X = data.iloc[:, 2:]
y = data.iloc[:, 0]


# Initialize performance tracking lists
train_performance = []
test_performance = []
all_test_preds = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
round_id = 1

smote = SMOTE(random_state=42)

# Loop through each fold
for train_index, test_index in skf.split(X, y):
    print(f"\nRound {round_id}")
    round_id += 1

    # Split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to training data
    X_train_r, y_train_r = smote.fit_resample(X_train, y_train)

    # Model training
    model = TabPFNClassifier(device="cuda")
    model.fit(X_train_r, y_train_r)

    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Evaluate performance
    train_acc = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train, average='weighted')

    test_acc = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted')

    train_performance.append({'Accuracy': train_acc, 'F1': train_f1})
    test_performance.append({'Accuracy': test_acc, 'F1': test_f1})

    print("Train Accuracy:", round(train_acc, 3), "F1:", round(train_f1, 3))
    print("Test  Accuracy:", round(test_acc, 3), "F1:", round(test_f1, 3))

    # Collect predictions
    fold_result = pd.DataFrame({
        'true_label': y_test.values,
        'predicted_label': y_pred_test,
        'group': data.iloc[test_index]['group'].values
    })
    all_test_preds.append(fold_result)


Round 1


tabpfn-v2-classifier-finetuned-zk73skhh.(…):   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Train Accuracy: 0.995 F1: 0.995
Test  Accuracy: 0.787 F1: 0.787

Round 2
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.82 F1: 0.81

Round 3
Train Accuracy: 0.991 F1: 0.991
Test  Accuracy: 0.672 F1: 0.65

Round 4
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.721 F1: 0.714

Round 5
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.803 F1: 0.805

Round 6
Train Accuracy: 0.993 F1: 0.993
Test  Accuracy: 0.721 F1: 0.702

Round 7
Train Accuracy: 0.998 F1: 0.998
Test  Accuracy: 0.721 F1: 0.693

Round 8
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.75 F1: 0.728

Round 9
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.633 F1: 0.618

Round 10
Train Accuracy: 0.996 F1: 0.996
Test  Accuracy: 0.783 F1: 0.755


In [6]:
# save all test predictions
final_result = pd.concat(all_test_preds, ignore_index=True)
final_result.to_csv(FIANL_RESULTS_PATH, index=False)

# Calculate average performance
df_train_perf = pd.DataFrame(train_performance)
df_test_perf = pd.DataFrame(test_performance)

print("\nAverage Train Performance:")
for metric in df_train_perf.columns:
    print(f"{metric:<10}: {df_train_perf[metric].mean():.3f} ± {df_train_perf[metric].std():.3f}")

print("\nAverage Test Performance:")
for metric in df_test_perf.columns:
    print(f"{metric:<10}: {df_test_perf[metric].mean():.3f} ± {df_test_perf[metric].std():.3f}")


Average Train Performance:
Accuracy  : 0.995 ± 0.002
F1        : 0.995 ± 0.002

Average Test Performance:
Accuracy  : 0.741 ± 0.059
F1        : 0.726 ± 0.064


In [7]:
# Train final model on entire dataset
final_model = TabPFNClassifier(device="cuda")
final_model.fit(X, y)

# Save as pickle file
with open(MODEL_PATH, "wb") as f:
    pickle.dump(final_model, f)